# Establishing a Connection - Alice
## Role: Inviter

In this notebook we'll be initiating the aries DID Exchange protocol. Before an agent can begin to interact with another agent they must engage in this protocol resulting in the exchange of Decentralised Identifiers (DIDs) and corresponding DID Documents. The information within these documents can then be used to establish a secure communication channel between the parties.

For a detailed description of this protocol it is highly recommened to read this [aries-rfc]((https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange).

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's notebook

<b>Carry on in [Bob's notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>
    
11. Accept Request for Connection
12. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [6]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [7]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)


### 3. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [14]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Task exception was never retrieved
future: <Task finished name='Task-23' coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:173> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 179, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.8/site-packages/aiohttp/web_runner.py", line 121, in start
    self._server = await loop.create_server(
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 1463, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [13]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

ClientResponseError: 401, message='Unauthorized', url=URL('http://alice-agent:8021/connections/create-invitation')

Error during POST /connections/create-invitation: 401, message='Unauthorized', url=URL('http://alice-agent:8021/connections/create-invitation')


### 5. Copy the invitation output from 4 and move over to the [Bob notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) 

### 11. Accept Request for Connection

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

### 12. Send Trust Ping to activate the conneciton

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

####  12.1 Check if connection is active

In [ ]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [8]:
response = await agent_controller.terminate()
print(response)

None
